In [36]:
from getpass import getpass
from mysql.connector import connect, Error
import pandas as pd
import numpy as np
import datetime
import re
import os
import time
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
# получить дату предыдущей операции

try:
    with connect(
        host="localhost",
        user="root",
        password="Gtxtymr@21",
        database='daily_operations'
    ) as connection:
        query = """SELECT client_id, operation_sum, operation_datetime, MONTH_diff FROM (WITH unique_id AS (SELECT DISTINCT client_id FROM voo_operations WHERE operation_date BETWEEN "2020-04-01" AND "2021-04-30" AND client_id IS NOT NULL),
all_operations AS (SELECT client_id, COUNT(operation_datetime) cnt, operation_datetime AS date_date FROM voo_operations WHERE client_id IN (SELECT * FROM unique_id) GROUP BY 1, 3 ORDER BY 1, 3 DESC)

SELECT voo_operations.client_id, voo_operations.operation_sum, voo_operations.operation_datetime, all_operations.date_date, 
ABS(TIMESTAMPDIFF(MONTH, voo_operations.operation_datetime, all_operations.date_date)) AS MONTH_diff,
ROW_NUMBER() over (PARTITION BY voo_operations.client_id, voo_operations.operation_datetime ORDER BY all_operations.date_date DESC) AS number_row FROM voo_operations
LEFT JOIN all_operations
ON  voo_operations.client_id = all_operations.client_id AND all_operations.date_date < voo_operations.operation_datetime

WHERE voo_operations.operation_date BETWEEN "2020-04-01" AND "2021-04-30") end_table
WHERE number_row = 1 AND client_id IS NOT NULL
ORDER BY 1, 3"""
        df = pd.read_sql(query, connection)
except Error as e:
    print(e)

In [ ]:
# возьмем те же операции и добавим туда филиал, если в предыдущий добавить филиал то почему то сервак кладет



try:
    with connect(
        host="localhost",
        user="root",
        password="Gtxtymr@21",
        database='daily_operations'
    ) as connection:
        query = """SELECT client_id, department, operation_datetime FROM voo_operations WHERE operation_date BETWEEN "2020-04-01" AND "2021-04-30" AND client_id IS NOT NULL
ORDER BY 1, 3;"""
        df = pd.read_sql(query, connection)
except Error as e:
    print(e)

In [62]:
# лояльность отделению


try:
    with connect(
        host="localhost",
        user="root",
        password="Gtxtymr@21",
        database='daily_operations'
    ) as connection:
        query = """SELECT client_id, department, COUNT(operation_date) Кол_во_операций FROM voo_operations WHERE operation_date BETWEEN "2020-04-01" AND "2021-04-30"
AND client_id IS NOT NULL
GROUP BY 1, 2;
"""
        df2 = pd.read_sql(query, connection)
except Error as e:
    print(e)

In [40]:
df2_2 = df2.groupby(['client_id', 'department'])['Кол_во_операций'].sum()


In [41]:
df2_group = df2_2.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))

In [59]:
df2_group.unstack().T.mean(axis=1).sort_values()

department
Дополнительный офис № 2 "Московский" по г. Москве    80.332165
ОКВКУ № 24 "Тульская"                                80.390627
ОКВКУ № 18 "Люблино"                                 81.503817
Дополнительный офис № 4 "Пресня"                     81.702361
ОКВКУ № 6 "Красные ворота"                           82.083159
Дополнительный офис № 1                              82.368196
ОКВКУ № 23 "Чертаново"                               82.757564
ОКВКУ № 8 "Павелецкая"                               82.896432
ОКВКУ № 21 "Кузьминки"                               83.001823
ОКВКУ № 22 "Полежаевская"                            83.131631
ОКВКУ № 10 "Белорусская"                             83.228553
ОКВКУ № 20 "Войковская"                              83.441997
ОКВКУ № 11 "Новые Черемушки"                         83.911793
ОКВКУ № 19 "Заубер Сити"                             84.291491
ОКВКУ № 15 "Спортивная"                              84.579859
Головное отделение                          

In [73]:
df2_3 = df2.groupby('client_id').agg({'department': lambda x: ' - '.join([i for i in sorted(x)]) if len(x) in (2, 3) else 0})

In [ ]:
df2_3['department'].value_counts()[1:50]

In [2]:
query = """SELECT DISTINCT client_phone FROM voo_clients
JOIN voo_operations ON voo_operations.client_id = voo_clients.client_id
JOIN departments
ON departments.department = voo_operations.department
WHERE departments.city = "Москва" AND client_phone IS NOT NULL"""



try:
    with connect(
        host="localhost",
        user="root",
        password="Gtxtymr@21",
        database='daily_operations'
    ) as connection:
        df2 = pd.read_sql(query, connection)
except Error as e:
    print(e)

In [6]:
df2['Длина'] = df2.client_phone.astype(str).str.len()

In [8]:
def number(x):
    x = str(x)
    if len(x) > 11:
        x = x[:11]
    if not all([i.isdigit() for i in x]):
        return 'ОШИБКА'
    return x

df2['Номера'] = df2.client_phone.apply(number)

In [9]:
df2.to_excel('Телефоны_Москва.xlsx')

In [10]:
df2

client_phone  Длина       Номера
0         79265883204     11  79265883204
1         79032078389     11  79032078389
2         79164859516     11  79164859516
3         79166311101     11  79166311101
4         79265509825     11  79265509825
...               ...    ...          ...
222127  79168577270.0     13  79168577270
222128    79057705113     11  79057705113
222129    79520637973     11  79520637973
222130    79253097001     11  79253097001
222131    79067292668     11  79067292668

[222132 rows x 3 columns]

In [2]:
query = """WITH cards_msk AS
(SELECT DISTINCT card_number FROM voo_operations JOIN departments USING(department) WHERE departments.city = "Москва" AND card_number IS NOT NULL),
cards_spb AS 
(SELECT DISTINCT card_number FROM voo_operations JOIN departments USING(department) WHERE departments.city = "Санкт-Петербург" AND card_number IS NOT NULL)
SELECT DISTINCT card_number,
IF(card_number IN (SELECT * FROM cards_msk), 1, 0) Москва, 
IF(card_number IN (SELECT * FROM cards_spb), 1, 0) Санкт_Петербург,
IF(card_number IN (SELECT * FROM cards_spb) AND card_number IN (SELECT * FROM cards_msk), 1, 0) МСК_СПБ
FROM voo_operations
WHERE card_number IS NOT NULL"""



try:
    with connect(
        host="localhost",
        user="root",
        password="Gtxtymr@21",
        database='daily_operations'
    ) as connection:
        df2 = pd.read_sql(query, connection)
except Error as e:
    print(e)

In [12]:
df = pd.read_excel(r'D:\FTP-DATA\Temp\Users_cards\Пользователи приложения.xlsx')

In [8]:
def city(row):
    if row['МСК_СПБ']:
        return 'Москва/Санкт-Петербург'
    elif row['Москва']:
        return 'Москва'
    elif row['Санкт_Петербург']:
        return 'Санкт-Петербург'
 
df2['Город'] = df2.apply(city, axis=1)
    

In [10]:
df2['Город'].value_counts()

Санкт-Петербург           19087
Москва                    11119
Москва/Санкт-Петербург      222
Name: Город, dtype: int64

In [18]:
df['Номер карты привилегий'].dtypes

dtype('int64')

In [28]:
df['Номер карты привилегий'] = df['Номер карты привилегий'].astype(int)

In [30]:
df2['card_number'] = df2['card_number'].astype(float).astype(int)

In [31]:
df = df.merge(df2[['card_number', 'Город']], how='left', left_on='Номер карты привилегий', right_on='card_number')

In [34]:
df.to_excel(r'D:\FTP-DATA\Temp\Users_cards\Пользователи_приложения_с_городом.xlsx', index=False)

In [35]:
df

№ сессии        Время начала Время последней активности  Время конца  \
0         5864 2021-01-18 17:02:37        2021-05-11 11:48:36          NaN   
1        15822 2021-05-13 15:04:58        2021-05-13 15:10:09          NaN   
2         2274 2020-10-21 18:10:09        2021-05-18 18:29:05          NaN   
3        11251 2021-03-11 11:24:56        2021-05-13 14:02:37          NaN   
4        13674 2021-04-12 16:22:06        2021-04-22 12:56:56          NaN   
...        ...                 ...                        ...          ...   
6898     16301 2021-05-21 14:58:32        2021-05-21 15:24:37          NaN   
6899     16134 2021-05-19 14:24:45        2021-05-21 15:05:20          NaN   
6900     16302 2021-05-21 15:05:12        2021-05-21 15:12:11          NaN   
6901      7474 2021-02-11 15:00:41        2021-05-21 15:43:09          NaN   
6902      7011 2021-02-04 17:10:26        2021-05-21 15:48:26          NaN   

     Имя сессии Платформа Версия клиента  Номер карты привилегий  \
0           NaN   ANDROID     2002-06-01                   10042   
1           NaN       IOS     2002-06-01                   10117   
2           NaN       IOS     2002-06-01                   10128   
3           NaN       IOS     2002-06-01                   10164   
4           NaN       IOS     2002-06-01                   10241   
...         ...       ...            ...                     ...   
6898        NaN       NaN     2002-06-01                 9005222   
6899        NaN       IOS     2002-06-01                 9005223   
6900        NaN   ANDROID     2002-06-01                 9005224   
6901        NaN   ANDROID     2002-06-01                 9005225   
6902        NaN       IOS     2002-06-01                 9005226   

      Карта привилегий  Номер телефона                  Эл. почта     Фамилия  \
0                10042    7.921953e+10                        NaN    Абакумов   
1                10117    7.911963e+10                        NaN      ЕКИМОВ   
2                10128    7.981871e+10                        NaN  БОГАТЕНКОВ   
3                10164    7.921939e+10                        NaN    РАТНИКОВ   
4                10241    7.963303e+10                        NaN    МОСИЕНКО   
...                ...             ...                        ...         ...   
6898           9005222    7.917595e+10                        NaN         Сас   
6899           9005223    7.926172e+10  elman.djafarov@icloud.com    Джафаров   
6900           9005224    7.926206e+10       aloevalo74@gmail.com   Агаммадов   
6901           9005225    7.985215e+10              Sii64@mail.ru     Соболев   
6902           9005226    7.981847e+10       nataliallerg@mail.ru     Ляшенко   

            Имя       Отчество        Дата рождения  Пол Рег. в прог. првлг.  \
0     Александр   Владимирович  1977-12-10 00:00:00  1.0          2016-12-22   
1         ПАВЕЛ     ВАЛЕРЬЕВИЧ  1976-05-25 00:00:00  1.0          2016-12-15   
2          ИЛЬЯ     ВАСИЛЬЕВИЧ  1994-07-24 00:00:00  1.0          2016-12-28   
3       АЛЕКСЕЙ  АЛЕКСАНДРОВИЧ  1974-11-22 00:00:00  1.0          2017-02-17   
4          НИНА    АНАТОЛЬЕВНА  1961-07-14 00:00:00  2.0          2017-04-06   
...         ...            ...                  ...  ...                 ...   
6898    Надежда       Ивановна  1979-02-27 00:00:00  2.0          2021-05-21   
6899     Эльман           Юсиф  1974-07-16 00:00:00  1.0          2021-05-21   
6900      Канан        Низами   1999-06-24 00:00:00  1.0          2021-05-21   
6901      Игорь       Иванович  1964-12-19 00:00:00  1.0          2021-05-21   
6902    Наталья     Леонидовна  1990-08-14 00:00:00  2.0          2021-05-21   

      card_number            Город  
0         10042.0  Санкт-Петербург  
1         10117.0  Санкт-Петербург  
2         10128.0  Санкт-Петербург  
3         10164.0  Санкт-Петербург  
4         10241.0  Санкт-Петербург  
...           ...              ...  
6898          NaN              NaN  
6899          NaN     

In [37]:
query = """SELECT voo_operations.department Отделение, YEAR(voo_operations.operation_date) Год, MONTH(voo_operations.operation_date) Месяц, voo_operations.operation_type Тип_операции,
SUM(voo_operations.operation_sum) Оборот, SUM(voo_operations.operation_revenue) Доход, COUNT(voo_operations.operation_id) Кол_во_операций
FROM voo_operations
WHERE voo_operations.operation_date >= "2020-06-01"
GROUP BY 1, 2, 3, 4;"""



try:
    with connect(
        host="localhost",
        user="root",
        password="Gtxtymr@21",
        database='daily_operations'
    ) as connection:
        df = pd.read_sql(query, connection)
except Error as e:
    print(e)

In [39]:
df.to_excel('Покупки_продажи.xlsx')